In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ Device: {device}")

print("="*70)
print("🧠 WEAK SIGNAL STRATEGY: PSEUDO-LABEL + PER-CLASS SELECTION")
print("="*70)

# ============================================================================
# LOAD DATA
# ============================================================================
print("\n[1/7] Loading data...")
train = pd.read_csv('//kaggle/input/thapar-ucs-654-2026-hack-01/train.csv')
test = pd.read_csv('/kaggle/input/thapar-ucs-654-2026-hack-01/test.csv')

TARGET_COL = 'target'
ID_COL = 'id'
feature_cols = [col for col in train.columns if col not in [TARGET_COL, ID_COL]]

y = train[TARGET_COL].values
num_classes = 20
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Remove duplicates
corr_matrix = train[feature_cols].corr()
features = corr_matrix.columns.tolist()
visited = set()
to_keep = []
for feat in features:
    if feat not in visited:
        to_keep.append(feat)
        for other in features:
            if other not in visited and abs(corr_matrix.loc[feat, other]) >= 0.999:
                visited.add(other)
        visited.add(feat)

X_raw = train[to_keep].values
X_test_raw = test[to_keep].values

print(f"   Train: {X_raw.shape}, Test: {X_test_raw.shape}")

# Feature importance for polynomial
rf_temp = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_temp.fit(X_raw, y)
importance_order = np.argsort(rf_temp.feature_importances_)

# ============================================================================
# CREATE MAIN FEATURES
# ============================================================================
print("\n[2/7] Creating features...")

def make_poly(X, X_test, top_k, interaction_only=True):
    X_top = X[:, top_k]
    X_test_top = X_test[:, top_k]
    poly = PolynomialFeatures(degree=2, interaction_only=interaction_only, include_bias=False)
    X_poly = poly.fit_transform(X_top)
    X_test_poly = poly.transform(X_test_top)
    X_full = np.hstack([X, X_poly])
    X_test_full = np.hstack([X_test, X_test_poly])
    scaler = StandardScaler()
    return scaler.fit_transform(X_full), scaler.transform(X_test_full)

# Main feature set
top_30 = importance_order[-30:]
X_main, X_test_main = make_poly(X_raw, X_test_raw, top_30)
print(f"   Main features: {X_main.shape[1]}")

# Alternative feature sets for diversity
top_25 = importance_order[-25:]
X_v2, X_test_v2 = make_poly(X_raw, X_test_raw, top_25)

top_35 = importance_order[-35:]
X_v3, X_test_v3 = make_poly(X_raw, X_test_raw, top_35)

# ============================================================================
# TRAIN BASE MODELS (Round 1)
# ============================================================================
print("\n[3/7] Training base models (Round 1)...")

all_oof = {}
all_test = {}

# --- SVM on different views ---
views = {
    'svm_v1': (X_main, X_test_main),
    'svm_v2': (X_v2, X_test_v2),
    'svm_v3': (X_v3, X_test_v3),
}

for name, (X_view, X_test_view) in views.items():
    print(f"\n   Training {name}...")
    oof = np.zeros((len(y), num_classes))
    test_pred = np.zeros((len(X_test_view), num_classes))
    
    for fold, (tr_idx, val_idx) in enumerate(cv.split(X_view, y)):
        svm = SVC(kernel='rbf', C=5, gamma='scale', probability=True, random_state=42)
        svm.fit(X_view[tr_idx], y[tr_idx])
        oof[val_idx] = svm.predict_proba(X_view[val_idx])
        test_pred += svm.predict_proba(X_test_view) / 5
    
    acc = accuracy_score(y, np.argmax(oof, axis=1))
    all_oof[name] = oof
    all_test[name] = test_pred
    print(f"   ✅ {name}: {acc*100:.2f}%")

# --- Gated Network ---
print(f"\n   Training Gated Network...")

class GatedBlock(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(dim, dim*2), nn.GELU(), nn.Dropout(0.3), nn.Linear(dim*2, dim))
        self.gate = nn.Sequential(nn.Linear(dim, dim), nn.Sigmoid())
        self.norm = nn.LayerNorm(dim)
    def forward(self, x):
        return self.norm(x + self.net(x) * self.gate(x))

class GatedNet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.gate = nn.Sequential(nn.Linear(input_dim, input_dim), nn.Sigmoid())
        self.embed = nn.Sequential(nn.Linear(input_dim, 256), nn.LayerNorm(256), nn.GELU(), nn.Dropout(0.3))
        self.b1, self.b2, self.b3 = GatedBlock(256), GatedBlock(256), GatedBlock(256)
        self.out = nn.Sequential(nn.Linear(256, 128), nn.GELU(), nn.Dropout(0.15), nn.Linear(128, num_classes))
    def forward(self, x):
        x = x * self.gate(x)
        x = self.b3(self.b2(self.b1(self.embed(x))))
        return self.out(x)

def train_nn(model_cls, X_tr, y_tr, X_val, y_val, epochs=150, patience=20):
    loader = DataLoader(TensorDataset(torch.FloatTensor(X_tr), torch.LongTensor(y_tr)), batch_size=256, shuffle=True)
    model = model_cls(X_tr.shape[1], num_classes).to(device)
    opt = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.02)
    sched = optim.lr_scheduler.CosineAnnealingWarmRestarts(opt, T_0=20, T_mult=2)
    crit = nn.CrossEntropyLoss(label_smoothing=0.1)
    best_acc, best_state, wait = 0, None, 0
    for _ in range(epochs):
        model.train()
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            crit(model(xb), yb).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
        sched.step()
        model.eval()
        with torch.no_grad():
            pred = torch.argmax(model(torch.FloatTensor(X_val).to(device)), dim=1).cpu().numpy()
        acc = accuracy_score(y_val, pred)
        if acc > best_acc:
            best_acc, best_state, wait = acc, model.state_dict().copy(), 0
        else:
            wait += 1
            if wait >= patience: break
    model.load_state_dict(best_state)
    return model

oof = np.zeros((len(y), num_classes))
test_pred = np.zeros((len(X_test_main), num_classes))
for fold, (tr_idx, val_idx) in enumerate(cv.split(X_main, y)):
    model = train_nn(GatedNet, X_main[tr_idx], y[tr_idx], X_main[val_idx], y[val_idx])
    model.eval()
    with torch.no_grad():
        oof[val_idx] = torch.softmax(model(torch.FloatTensor(X_main[val_idx]).to(device)), dim=1).cpu().numpy()
        test_pred += torch.softmax(model(torch.FloatTensor(X_test_main).to(device)), dim=1).cpu().numpy() / 5

acc = accuracy_score(y, np.argmax(oof, axis=1))
all_oof['gated'] = oof
all_test['gated'] = test_pred
print(f"   ✅ gated: {acc*100:.2f}%")
torch.cuda.empty_cache() if torch.cuda.is_available() else None

# ============================================================================
# PSEUDO-LABELING
# ============================================================================
print("\n[4/7] PSEUDO-LABELING (using test data!)...")

# Average predictions from all models
avg_test_proba = np.mean([all_test[n] for n in all_oof.keys()], axis=0)
avg_test_pred = np.argmax(avg_test_proba, axis=1)
avg_test_conf = np.max(avg_test_proba, axis=1)

# Select HIGH CONFIDENCE predictions (top 30%)
confidence_threshold = np.percentile(avg_test_conf, 70)
high_conf_mask = avg_test_conf >= confidence_threshold
n_pseudo = high_conf_mask.sum()

print(f"   Confidence threshold: {confidence_threshold:.3f}")
print(f"   High confidence samples: {n_pseudo} ({n_pseudo/len(avg_test_conf)*100:.1f}%)")

# Create pseudo-labeled dataset
X_pseudo = X_test_main[high_conf_mask]
y_pseudo = avg_test_pred[high_conf_mask]

# Combine with original training data
X_combined = np.vstack([X_main, X_pseudo])
y_combined = np.hstack([y, y_pseudo])

print(f"   Original train: {len(y)}, After pseudo-label: {len(y_combined)}")

# ============================================================================
# TRAIN WITH PSEUDO-LABELS (Round 2)
# ============================================================================
print("\n[5/7] Training with pseudo-labels (Round 2)...")

# Train SVM on combined data
print("\n   Training SVM with pseudo-labels...")
oof_pseudo = np.zeros((len(y), num_classes))
test_pseudo = np.zeros((len(X_test_main), num_classes))

for fold, (tr_idx, val_idx) in enumerate(cv.split(X_main, y)):
    # Use original train indices but train on combined data concept
    # Actually, let's train on full combined and validate on original OOF
    svm = SVC(kernel='rbf', C=5, gamma='scale', probability=True, random_state=42)
    
    # For this fold, add pseudo-labels to training
    X_tr_fold = np.vstack([X_main[tr_idx], X_pseudo])
    y_tr_fold = np.hstack([y[tr_idx], y_pseudo])
    
    svm.fit(X_tr_fold, y_tr_fold)
    oof_pseudo[val_idx] = svm.predict_proba(X_main[val_idx])
    test_pseudo += svm.predict_proba(X_test_main) / 5

acc_pseudo = accuracy_score(y, np.argmax(oof_pseudo, axis=1))
all_oof['svm_pseudo'] = oof_pseudo
all_test['svm_pseudo'] = test_pseudo
print(f"   ✅ SVM + Pseudo-labels: {acc_pseudo*100:.2f}%")

# Train Gated with pseudo-labels
print("\n   Training Gated with pseudo-labels...")
oof_gated_pseudo = np.zeros((len(y), num_classes))
test_gated_pseudo = np.zeros((len(X_test_main), num_classes))

for fold, (tr_idx, val_idx) in enumerate(cv.split(X_main, y)):
    X_tr_fold = np.vstack([X_main[tr_idx], X_pseudo])
    y_tr_fold = np.hstack([y[tr_idx], y_pseudo])
    
    model = train_nn(GatedNet, X_tr_fold, y_tr_fold, X_main[val_idx], y[val_idx], epochs=100, patience=15)
    model.eval()
    with torch.no_grad():
        oof_gated_pseudo[val_idx] = torch.softmax(model(torch.FloatTensor(X_main[val_idx]).to(device)), dim=1).cpu().numpy()
        test_gated_pseudo += torch.softmax(model(torch.FloatTensor(X_test_main).to(device)), dim=1).cpu().numpy() / 5

acc_gated_pseudo = accuracy_score(y, np.argmax(oof_gated_pseudo, axis=1))
all_oof['gated_pseudo'] = oof_gated_pseudo
all_test['gated_pseudo'] = test_gated_pseudo
print(f"   ✅ Gated + Pseudo-labels: {acc_gated_pseudo*100:.2f}%")

torch.cuda.empty_cache() if torch.cuda.is_available() else None

# ============================================================================
# PER-CLASS MODEL SELECTION (NOT STACKING!)
# ============================================================================
print("\n[6/7] PER-CLASS MODEL SELECTION...")

model_names = list(all_oof.keys())
print(f"\n   Models available: {model_names}")

# For each class, find which model has highest recall
per_class_best = {}
for c in range(num_classes):
    best_model = None
    best_recall = 0
    
    for name in model_names:
        # Get predictions for this class
        pred = np.argmax(all_oof[name], axis=1)
        true_mask = (y == c)
        if true_mask.sum() == 0:
            continue
        
        # Recall for this class
        recall = (pred[true_mask] == c).sum() / true_mask.sum()
        
        if recall > best_recall:
            best_recall = recall
            best_model = name
    
    per_class_best[c] = best_model
    print(f"   Class {c:2d}: {best_model} (recall={best_recall:.2f})")

# Create per-class ensemble prediction
def per_class_predict(oof_dict, test_dict, per_class_best, num_classes):
    """Select best model's probability for each class"""
    n_train = len(list(oof_dict.values())[0])
    n_test = len(list(test_dict.values())[0])
    
    oof_combined = np.zeros((n_train, num_classes))
    test_combined = np.zeros((n_test, num_classes))
    
    for c in range(num_classes):
        best_model = per_class_best[c]
        oof_combined[:, c] = oof_dict[best_model][:, c]
        test_combined[:, c] = test_dict[best_model][:, c]
    
    # Normalize to sum to 1
    oof_combined = oof_combined / (oof_combined.sum(axis=1, keepdims=True) + 1e-10)
    test_combined = test_combined / (test_combined.sum(axis=1, keepdims=True) + 1e-10)
    
    return oof_combined, test_combined

oof_perclass, test_perclass = per_class_predict(all_oof, all_test, per_class_best, num_classes)
acc_perclass = accuracy_score(y, np.argmax(oof_perclass, axis=1))
print(f"\n   ✅ Per-Class Selection: {acc_perclass*100:.2f}%")

# ============================================================================
# FINAL COMBINATIONS
# ============================================================================
print("\n[7/7] Final combinations...")

# Simple average of all
simple_avg = np.mean([all_oof[n] for n in model_names], axis=0)
simple_acc = accuracy_score(y, np.argmax(simple_avg, axis=1))
print(f"\n   Simple Average (all): {simple_acc*100:.2f}%")

# Average of pseudo-labeled models only
pseudo_models = ['svm_pseudo', 'gated_pseudo']
pseudo_avg = np.mean([all_oof[n] for n in pseudo_models], axis=0)
pseudo_avg_acc = accuracy_score(y, np.argmax(pseudo_avg, axis=1))
print(f"   Pseudo-label Average: {pseudo_avg_acc*100:.2f}%")

# Combine per-class with pseudo average
combined = 0.5 * oof_perclass + 0.5 * pseudo_avg
combined_acc = accuracy_score(y, np.argmax(combined, axis=1))
print(f"   PerClass + Pseudo: {combined_acc*100:.2f}%")

# Try different blends
print("\n   Searching blends...")
best_blend_acc = 0
best_blend = None
best_blend_test = None

for w1 in np.arange(0, 1.05, 0.1):
    for w2 in np.arange(0, 1.05 - w1, 0.1):
        w3 = 1 - w1 - w2
        if w3 < 0: continue
        
        blend = w1 * simple_avg + w2 * oof_perclass + w3 * pseudo_avg
        acc = accuracy_score(y, np.argmax(blend, axis=1))
        
        if acc > best_blend_acc:
            best_blend_acc = acc
            best_blend = blend
            best_blend_test = (w1 * np.mean([all_test[n] for n in model_names], axis=0) + 
                              w2 * test_perclass + 
                              w3 * np.mean([all_test[n] for n in pseudo_models], axis=0))

print(f"   Best Blend: {best_blend_acc*100:.2f}%")

# ============================================================================
# RESULTS
# ============================================================================
print("\n" + "="*70)
print("📊 FINAL RESULTS")
print("="*70)

results = {
    'Simple Average': simple_acc,
    'Per-Class Selection': acc_perclass,
    'Pseudo-label Average': pseudo_avg_acc,
    'PerClass + Pseudo': combined_acc,
    'Best Blend': best_blend_acc,
}

# Individual models
print("\n   Individual Models:")
for name in model_names:
    acc = accuracy_score(y, np.argmax(all_oof[name], axis=1))
    print(f"      {name}: {acc*100:.2f}%")

print("\n   Combination Methods:")
for name, acc in sorted(results.items(), key=lambda x: -x[1]):
    marker = "🏆" if acc == max(results.values()) else "  "
    print(f"   {marker} {name}: {acc*100:.2f}%")

# Pick best
best_method = max(results, key=lambda x: results[x])
best_acc = results[best_method]

# ============================================================================
# SUBMISSION
# ============================================================================
print("\n" + "="*70)
print("📁 GENERATING SUBMISSION")
print("="*70)

# Use best method
if best_method == 'Best Blend':
    final_test = best_blend_test
elif best_method == 'Per-Class Selection':
    final_test = test_perclass
elif best_method == 'Pseudo-label Average':
    final_test = np.mean([all_test[n] for n in pseudo_models], axis=0)
elif best_method == 'PerClass + Pseudo':
    final_test = 0.5 * test_perclass + 0.5 * np.mean([all_test[n] for n in pseudo_models], axis=0)
else:
    final_test = np.mean([all_test[n] for n in model_names], axis=0)

final_pred = np.argmax(final_test, axis=1).astype(int)

submission = pd.DataFrame({
    'id': test['id'].values,
    'target': final_pred
})

submission.to_csv('/kaggle/working/submission_pseudo_perclass.csv', index=False)
print(f"\n✅ Saved: submission_pseudo_perclass.csv")
print(f"   Method: {best_method}")
print(f"   OOF Accuracy: {best_acc*100:.2f}%")

print("\n" + "="*70)
if best_acc >= 0.28:
    print(f"✅ TARGET: {best_acc*100:.2f}% >= 28%")
else:
    print(f"📈 Best: {best_acc*100:.2f}%")
print("="*70)

🖥️ Device: cuda
🧠 WEAK SIGNAL STRATEGY: PSEUDO-LABEL + PER-CLASS SELECTION

[1/7] Loading data...
   Train: (10000, 50), Test: (10000, 50)

[2/7] Creating features...
   Main features: 515

[3/7] Training base models (Round 1)...

   Training svm_v1...
   ✅ svm_v1: 22.15%

   Training svm_v2...
   ✅ svm_v2: 22.63%

   Training svm_v3...
   ✅ svm_v3: 18.41%

   Training Gated Network...
   ✅ gated: 21.45%

[4/7] PSEUDO-LABELING (using test data!)...
   Confidence threshold: 0.229
   High confidence samples: 3000 (30.0%)
   Original train: 10000, After pseudo-label: 13000

[5/7] Training with pseudo-labels (Round 2)...

   Training SVM with pseudo-labels...
   ✅ SVM + Pseudo-labels: 26.98%

   Training Gated with pseudo-labels...
   ✅ Gated + Pseudo-labels: 27.41%

[6/7] PER-CLASS MODEL SELECTION...

   Models available: ['svm_v1', 'svm_v2', 'svm_v3', 'gated', 'svm_pseudo', 'gated_pseudo']
   Class  0: gated (recall=0.23)
   Class  1: svm_pseudo (recall=0.33)
   Class  2: gated_pseudo (r